# ACDtools dev sandbox 

#### Date: 11 October, 2024

Author = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

# Install ACDtools locally

In [1]:
# this needs to be set via a custom edit per user at the moment
!pip install --user -e /g/data/es60/users/thomas_moore/code/ACDtools

Obtaining file:///g/data/es60/users/thomas_moore/code/ACDtools
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ACDtools (pyproject.toml) ... done
  Created wheel for ACDtools: filename=ACDtools-0.1-0.editable-py3-none-any.whl size=3514 sha256=5c665ecaf9f2c8d0835a93c1c7f399475e500f83e44db9837d35a760a933d6ec
  Stored in directory: /jobfs/126952849.gadi-pbs/pip-ephem-wheel-cache-jgow2gu7/wheels/b6/a3/f2/6ce45fbdc116ad50e421d6a11cb060cc796e867501807af446
Successfully built ACDtools
  Attempting uninstall: ACDtools
    Found existing installation: ACDtools 0.1
    Uninstalling ACDtools-0.1:
      Successfully uninstalled ACDtools-0.1


In [3]:
# Enable autoreload in the notebook
%load_ext autoreload
%autoreload 1 
%aimport ACDtools.util
%aimport ACDtools.ard
# Importing from your local package util.py
from ACDtools.util import test_function
from ACDtools.util import detect_compute_platform
from ACDtools.util import load_config
from ACDtools.util import start_dask_cluster_from_config
from ACDtools.util import report_esm_unique
from ACDtools.util import var_name_info
from ACDtools.util import list_catalog_query_kwargs
from ACDtools.util import load_cmip6_fs38_datastore
# ard.py
from ACDtools.ard import load_ACCESS_ESM_ensemble
from ACDtools.ard import find_chunking_info

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Notebook settings

### filter warnings

In [ ]:
import warnings
warnings.filterwarnings("ignore") # Suppress warnings

# Dask cluster from config
`client, cluster = start_dask_cluster_from_config('netcdf_work')`
<br>OR<br>
`client, cluster = start_dask_cluster_from_config('zarr_work')`

In [ ]:
client, cluster = start_dask_cluster_from_config('netcdf_work')

# Issue: write function to load ACCESS-ESM1.5 data object using intake catalogs at NCI
- https://github.com/Thomas-Moore-Creative/ACDtools/issues/1

## utilise CMIP6 data catalogs for NCI holdings

##### Information on climate data catalogs across Australian HPC

**ACCESS-NRI** https://access-nri-intake-catalog.readthedocs.io/en/latest/usage/how.html <br>
**NCI** https://opus.nci.org.au/pages/viewpage.action?pageId=213713098


##### $\bigstar$ Get inspiration from ACCESS-NRI intake catalog docs: ACCESS-ESM1-5 CMIP6 example
https://access-nri-intake-catalog.readthedocs.io/en/latest/usage/quickstart.html

# import packages

In [ ]:
import intake
import xarray as xr
import numpy as np
import gc
import json

### import the ACCESS-NRI catalog

In [ ]:
catalog = intake.cat.access_nri

### (1) "I know I want Australian CMIP6 data - so that's fs38 and I need access to that NCI project"

In [36]:
cmip6_fs38_datastore = catalog.search(name='cmip6_fs38').to_source()

### (2) "what are the realms covered by cmip6_fs38?"

In [37]:
report_esm_unique(cmip6_fs38_datastore,keep_list=['realm'])

╒════════════╤═════════════════╕
│ Category   │ Unique values   │
╞════════════╪═════════════════╡
│ realm      │ aerosol         │
│            │ atmos           │
│            │ land            │
│            │ landIce         │
│            │ ocean           │
│            │ ocnBgchem       │
│            │ seaIce          │
╘════════════╧═════════════════╛


### (3) I want to see what variables, over what frequencies, are available in both the 'ocean' & 'oceanBgchem' realms

In [38]:
cmip6_fs38_ocean_datastore = cmip6_fs38_datastore.search(realm=['ocean','ocnBgchem'])

In [39]:
[sorted_unique_dict, table_data] = report_esm_unique(cmip6_fs38_ocean_datastore,return_results=True)

╒════════════════╤═══════════════════════╕
│ Category       │ Unique values         │
╞════════════════╪═══════════════════════╡
│ experiment_id  │ 1pctCO2               │
│                │ 1pctCO2-bgc           │
│                │ 1pctCO2-cdr           │
│                │ 1pctCO2-rad           │
│                │ abrupt-4xCO2          │
│                │ esm-1pct-brch-1000PgC │
│                │ esm-1pct-brch-2000PgC │
│                │ esm-1pct-brch-750PgC  │
│                │ esm-hist              │
│                │ esm-pi-CO2pulse       │
│                │ esm-pi-cdr-pulse      │
│                │ esm-piControl         │
│                │ esm-ssp585            │
│                │ esm-ssp585-ssp126Lu   │
│                │ faf-all               │
│                │ faf-heat              │
│                │ faf-heat-NA0pct       │
│                │ faf-heat-NA50pct      │
│                │ faf-passiveheat       │
│                │ faf-stress            │
│          

# what is the long name of a particular variable?

In [40]:
var_name_info(cmip6_fs38_ocean_datastore,'intpp')

*** Variable: intpp from catalog: <cmip6-fs38 catalog with 97270 dataset(s) from 414120 asset(s)> ***
╒════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════╕
│ Attribute      │ Value                                                                                               │
╞════════════════╪═════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ standard_name  │ net_primary_mole_productivity_of_biomass_expressed_as_carbon_by_phytoplankton                       │
├────────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ long_name      │ Primary Organic Carbon Production by All Types of Phytoplankton                                     │
├────────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ comment        │ Vertically integrated total prim

# filter catalog for final ACCESS-ESM1.5 dataset

In [41]:
final_search = cmip6_fs38_ocean_datastore.search(file_type='l',
                    variable_id='intpp',source_id='ACCESS-ESM1-5',experiment_id='historical')

In [42]:
report_esm_unique(final_search)

╒════════════════╤═════════════════╕
│ Category       │ Unique values   │
╞════════════════╪═════════════════╡
│ experiment_id  │ historical      │
├────────────────┼─────────────────┤
│ file_type      │ l               │
├────────────────┼─────────────────┤
│ frequency      │ mon             │
├────────────────┼─────────────────┤
│ grid_label     │ gn              │
├────────────────┼─────────────────┤
│ institution_id │ CSIRO           │
├────────────────┼─────────────────┤
│ project_id     │ CMIP            │
├────────────────┼─────────────────┤
│ realm          │ ocnBgchem       │
├────────────────┼─────────────────┤
│ source_id      │ ACCESS-ESM1-5   │
├────────────────┼─────────────────┤
│ table_id       │ Omon            │
├────────────────┼─────────────────┤
│ variable_id    │ intpp           │
╘════════════════╧═════════════════╛


# what is the chunking of the files in this final_search catalog?

In [ ]:
final_search.df['path'].iloc[0]

In [ ]:
find_chunking_info(final_search,'intpp',return_results=False)

# load without specifying any chunking

In [ ]:
%%time
ds_ESM15_esorted = load_ACCESS_ESM_ensemble(final_search)

In [ ]:
ds_ESM15_esorted

#### One still needs to know what dimensions (1, 300, 360 ; ) refers to and something about MB size per chunk to set the time to 220 . . . these rules of thumb should be in the yaml settings file until much more complicated heuristics could be coded

In [ ]:
%%time
ds_ESM15_esorted = load_ACCESS_ESM_ensemble(final_search,chunking_settings={'chunks':{'member':1,'time':220,'j':300,'i':360}})

In [ ]:
ds_ESM15_esorted

In [ ]:
%%time
ds_ESM15_esorted = load_ACCESS_ESM_ensemble(final_search,chunking_key='ACCESS_ESM15_2D')

In [ ]:
ds_ESM15_esorted

In [ ]:
ds_ESM15_esorted.isel(member=0).mean('time').intpp.plot()

# 3D dataset?

In [ ]:
thetao_search = cmip6_fs38_ocean_datastore.search(file_type='l',
                    variable_id='thetao',source_id='ACCESS-ESM1-5',experiment_id='historical')

In [ ]:
report_esm_unique(thetao_search)

In [ ]:
find_chunking_info(thetao_search,'thetao',return_results=False)

In [ ]:
find_chunking_info(thetao_search,'thetao',return_results=True)

In [ ]:
xr.open_mfdataset('/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r3i1p1f1/Omon/thetao/gn/v20191203/thetao_Omon_ACCESS-ESM1-5_historical_r3i1p1f1_gn_189001-189912.nc')

In [ ]:
%%time
ds_ESM15_esorted = load_ACCESS_ESM_ensemble(thetao_search)

In [ ]:
ds_ESM15_esorted

In [ ]:
%%time
ds_ESM15_esorted = load_ACCESS_ESM_ensemble(thetao_search,chunking_key='ACCESS_ESM15_3D')

In [ ]:
ds_ESM15_esorted

# let's use the tools as they exist to try to start the workflow

## I want Australian CMIP6 data

In [4]:
cmip6_fs38_datastore = load_cmip6_fs38_datastore()

In [13]:
cmip6_fs38_datastore

,unique
path,1054133
file_type,2
realm,7
frequency,10
table_id,24
project_id,1
institution_id,3
source_id,4
experiment_id,52
member_id,80


In [21]:
report_esm_unique(cmip6_fs38_datastore)

╒════════════════╤═══════════════════════╕
│ Category       │ Unique values         │
╞════════════════╪═══════════════════════╡
│ experiment_id  │ 1pctCO2               │
│                │ 1pctCO2-bgc           │
│                │ 1pctCO2-cdr           │
│                │ 1pctCO2-rad           │
│                │ abrupt-4xCO2          │
│                │ amip                  │
│                │ esm-1pct-brch-1000PgC │
│                │ esm-1pct-brch-2000PgC │
│                │ esm-1pct-brch-750PgC  │
│                │ esm-hist              │
│                │ esm-pi-CO2pulse       │
│                │ esm-pi-cdr-pulse      │
│                │ esm-piControl         │
│                │ esm-ssp585            │
│                │ esm-ssp585-ssp126Lu   │
│                │ faf-all               │
│                │ faf-heat              │
│                │ faf-heat-NA0pct       │
│                │ faf-heat-NA50pct      │
│                │ faf-passiveheat       │
│          

In [24]:
query_dict = {'realm':['ocean','ocnBgchem'],'source_id':'ACCESS-ESM1-5'}

In [25]:
cmip6_fs38_datastore.search(**query_dict)

,unique
path,342610
file_type,2
realm,2
frequency,4
table_id,5
project_id,1
institution_id,1
source_id,1
experiment_id,35
member_id,80


In [32]:
load_config()['catalog_search_query_dict']['ACCESS_ESM15']['all_ocean']

{'realm': ['ocean', 'ocnBgchem'], 'source_id': 'ACCESS-ESM1-5'}

In [38]:
report_esm_unique(cmip6_fs38_datastore.search(**load_config()['catalog_search_query_dict']['ACCESS_ESM15']['all_ocean']))

╒════════════════╤═══════════════════════╕
│ Category       │ Unique values         │
╞════════════════╪═══════════════════════╡
│ experiment_id  │ 1pctCO2               │
│                │ 1pctCO2-bgc           │
│                │ 1pctCO2-cdr           │
│                │ 1pctCO2-rad           │
│                │ abrupt-4xCO2          │
│                │ esm-1pct-brch-1000PgC │
│                │ esm-1pct-brch-2000PgC │
│                │ esm-1pct-brch-750PgC  │
│                │ esm-hist              │
│                │ esm-pi-CO2pulse       │
│                │ esm-pi-cdr-pulse      │
│                │ esm-piControl         │
│                │ esm-ssp585            │
│                │ esm-ssp585-ssp126Lu   │
│                │ hist-GHG              │
│                │ hist-aer              │
│                │ hist-bgc              │
│                │ hist-nat              │
│                │ hist-noLu             │
│                │ historical            │
│          

In [42]:
report_esm_unique(cmip6_fs38_datastore.search(**load_config()['catalog_search_query_dict']['ACCESS_ESM15']['CSEPTA']))

╒════════════════╤═════════════════╕
│ Category       │ Unique values   │
╞════════════════╪═════════════════╡
│ experiment_id  │ historical      │
│                │ piControl       │
│                │ ssp126          │
│                │ ssp370          │
│                │ ssp585          │
├────────────────┼─────────────────┤
│ file_type      │ l               │
├────────────────┼─────────────────┤
│ frequency      │ mon             │
├────────────────┼─────────────────┤
│ grid_label     │ gn              │
├────────────────┼─────────────────┤
│ institution_id │ CSIRO           │
├────────────────┼─────────────────┤
│ project_id     │ CMIP            │
├────────────────┼─────────────────┤
│ realm          │ ocnBgchem       │
├────────────────┼─────────────────┤
│ source_id      │ ACCESS-ESM1-5   │
├────────────────┼─────────────────┤
│ table_id       │ Omon            │
├────────────────┼─────────────────┤
│ variable_id    │ intpp           │
╘════════════════╧═════════════════╛
